<a href="https://colab.research.google.com/github/oleksandr-kushnir/Coursera_Capstone/blob/master/Assignment_Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Segmenting and Clustering Neighborhoods in Toronto

##1. Cleaning and Preparing the Neighborhoods Data

At first let's import the Pandas library

In [0]:
import pandas as pd

Now we can read the table from the Wikipedia article URL using Pandas native method '.read_html'

In [130]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(wiki_url)[0]
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


The table contains 'Not assigned' values in the 'Borough' column, let's get rid of this rows:

In [131]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Are there any 'Not assigned' values in the 'Neghborhood' column?

In [132]:
df[df['Neighborhood'] == 'Not assigned']

,Postal Code,Borough,Neighborhood


Let's check, if there are any NaN values in the DataFrame left:

In [133]:
df[df.isna().any(axis=1)]

,Postal Code,Borough,Neighborhood


The table contains now all the data in the right format, let's look how many rows it has:

In [134]:
print('Number of rows in the dataset:', df.shape[0])

Number of rows in the dataset: 103


##2. Getting the Geospatial Data

We read the geospatial data from the given URL and create a DataFrame:

In [135]:
geo_df = pd.read_csv('https://cocl.us/Geospatial_data')
geo_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Then we join them on our neighborhood DataFrame:

In [136]:
df_merged = df.join(geo_df.set_index('Postal Code'), on='Postal Code')
df_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


##3. Explore and cluster the neighborhoods in Toronto
At first we import all the necessary libraries

In [137]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Lets create a DataFrame of the only boroughs containing the word 'Toronto'

In [138]:
df_toronto = df_merged[df_merged['Borough'].str.contains('Toronto', regex=False)]
df_toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


Let's create a map of Toronto and our negborhoods on it:

In [139]:
# point in the middle of our neighborhoods
lat_center = df_toronto['Latitude'].mean()
long_center = df_toronto['Longitude'].mean()

#create map object with center in the middle of neighborhoods
map_toronto = folium.Map(location=[lat_center, long_center], zoom_start=10)

# mark all neighborhoods on the map
for lat, lon, bor, neigh in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
  label = 'Borough: \n{}\n, Neighborhood(s): \n{}\n'.format(bor, neigh)
  folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)


map_toronto

Define Foursquare Credentials and Version



In [0]:
#@title
CLIENT_ID = 'L1MM1ECQMJI4EWWW0SA2YOKK2D4PI2A1NHENJX2B5J3GAQLX' # your Foursquare ID
CLIENT_SECRET = 'KL1DMABXU2P2FMB5HPGJ3FSD2SE2KXULSRHKPDSCVP4S3HI2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100

Let's build a dataframe with vanues and their categories for each neighborhood:

In [141]:
data = []
for row in df_toronto.iterrows():
  ZIP, bor, neigh, lat, lng = row[1]

  # create the API request URL
  url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
              CLIENT_ID, 
              CLIENT_SECRET, 
              VERSION, 
              lat, 
              lng, 
              radius, 
              LIMIT)
  try:
    results = requests.get(url).json()["response"]['groups'][0]['items']

    # return only relevant information for each nearby venue
    for v in results:
      #building DataFrame 
      venue = v['venue']['name']
      category = v['venue']['categories'][0]['name'] 
      data_row = {'Postal Code': ZIP, 'Neighborhood(s)': neigh,	'Neighborhoods Latitude': lat,	'Neighborhoods Longitude': lng, 'Venue': venue, 'Venue Category': category}
      data.append(data_row)
  except:
    #skip neighborhoods without information
    continue 
df_venues = pd.DataFrame(data)
df_venues

,Postal Code,Neighborhood(s),Neighborhoods Latitude,Neighborhoods Longitude,Venue,Venue Category
0,M5A,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,Bakery
1,M5A,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,Coffee Shop
2,M5A,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,Breakfast Spot
3,M5A,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,Distribution Center
4,M5A,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,Spa
...,...,...,...,...,...,...
1597,M7Y,Business reply mail Processing Centre,43.662744,-79.321558,Jonathan Ashbridge Park,Park
1598,M7Y,Business reply mail Processing Centre,43.662744,-79.321558,Olliffe On Queen,Butcher
1599,M7Y,Business reply mail Processing Centre,43.662744,-79.321558,Greenwood Cigar & Variety,Smoke Shop
1600,M7Y,Business reply mail Processing Centre,43.662744,-79.321558,Revolution Recording,Recording Studio


Let's find out how many unique categories can be curated from all the returned venues:

In [142]:
print('Unique categories:', df_venues['Venue Category'].unique().shape[0])

Unique categories: 235


Let's create a DataFrame with one hot encoding for the category of each venue:

In [143]:
venues_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")
venues_onehot[['Neighborhoods Latitude',	'Neighborhoods Longitude', 'Neighborhood(s)']] = df_venues[['Neighborhoods Latitude',	'Neighborhoods Longitude', 'Neighborhood(s)']]

# move neighborhood column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]

venues_onehot

,Neighborhood(s),Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,...,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Neighborhoods Latitude,Neighborhoods Longitude
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43.654260,-79.360636
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43.654260,-79.360636
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43.654260,-79.360636
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43.654260,-79.360636
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43.654260,-79.360636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,Business reply mail Processing Centre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43.662744,-79.321558
1598,Business reply mail Processing Centre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43.662744,-79.321558
1599,Business reply mail Processing Centre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43.662744,-79.321558
1600,Business reply mail Processing Centre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43.662744,-79.321558


### Clustering
Let's prepare the dataset for clustering groping it by neighborhoods and calculating the mean for each category

In [144]:
venue_cluster = venues_onehot.groupby(by='Neighborhood(s)', as_index=False).mean()
venue_cluster

,Neighborhood(s),Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,...,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Neighborhoods Latitude,Neighborhoods Longitude
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.0000,0.017857,0.035714,0.000000,0.000000,0.00,0.017857,0.017857,0.000000,0.035714,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,...,0.000000,0.017857,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.00,0.000,0.017857,0.000000,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.00,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,43.644771,-79.373306
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.040000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.040000,43.636847,-79.428191
2,Business reply mail Processing Centre,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,...,0.000000,0.000000,0.058824,0.00,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,43.662744,-79.321558
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,43.628947,-79.394420
4,Central Bay Street,0.000000,0.000000,0.000000,0.00000

Now run *k*-means to cluster the neighborhood into 5 clusters and show the output labels

In [145]:
# set the number of clusters
n_clusters = 5

# run the clustering algorythm
kmeans = KMeans(n_clusters=n_clusters, random_state=1).fit(venue_cluster.drop(['Neighborhoods Latitude', 'Neighborhoods Longitude', 'Neighborhood(s)'], axis=1))

# check cluster labels
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 4, 0, 1, 0,
       0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's label the dataset

In [146]:
venue_cluster['Label'] = kmeans.labels_
venue_cluster

,Neighborhood(s),Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,...,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Neighborhoods Latitude,Neighborhoods Longitude,Label
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.0000,0.017857,0.035714,0.000000,0.000000,0.00,0.017857,0.017857,0.000000,0.035714,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,...,0.017857,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.00,0.000,0.017857,0.000000,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.00,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,43.644771,-79.373306,0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.040000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.040000,43.636847,-79.428191,0
2,Business reply mail Processing Centre,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,...,0.000000,0.058824,0.00,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,43.662744,-79.321558,0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,43.628947,-79.394420,0
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

Finaly, let's plot the resulting clusters on the map

In [226]:
 # point in the middle of our neighborhoods
lat_center1 = venue_cluster['Neighborhoods Latitude'].mean()
long_center1 = venue_cluster['Neighborhoods Longitude'].mean()

#create map object with center in the middle of neighborhoods
map_clusters = folium.Map(location=[lat_center1, long_center1], zoom_start=10.5)

# mark all neighborhoods on the map
colors = ['green', 'blue', 'yellow', 'brown', 'red', 'black', 'magenta', 'cyan', 'white', 'pink']
for lat, lon, neigh, color in zip(venue_cluster['Neighborhoods Latitude'], venue_cluster['Neighborhoods Longitude'], venue_cluster['Neighborhood(s)'], venue_cluster['Label']):
  label = 'Neighborhood(s): \n{}\n'.format(neigh)
  folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[color],
        fill=True,
        fill_opacity=0.5,
        parse_html=False).add_to(map_clusters)


map_clusters

Let's find out the main 5 venue categories for each cluster

In [229]:
# let's group neighborhoods by clusters
neighborhoods_clustered = []
for i in range(5):
   cluster_list = venue_cluster[venue_cluster['Label'] == i]['Neighborhood(s)'].to_list()
   neighborhoods_clustered.append(cluster_list)
neighborhoods_clustered

#and have a look on top 5 categories for each cluster

#list of categories of each cluster
categories_cl_list = []
for cluster in neighborhoods_clustered:
  #categories for each cluster
  categories_cl = pd.Series()
  for neighborhood in cluster:
    # categories for each neighborhood
    neighborhood_cat  = df_venues[df_venues['Neighborhood(s)'] == neighborhood]['Venue Category']
    categories_cl = categories_cl.append(neighborhood_cat)
  categories_cl_list.append(categories_cl)

for label in range(5):
  print('Label', label, ':', colors[label])
  print('Top-5 Categories:\n', categories_cl_list[label].value_counts()[:5].index.to_list())
  print('----------------------------------------------------------------------------------')


Label 0 : green
Top-5 Categories:
 ['Coffee Shop', 'Café', 'Restaurant', 'Italian Restaurant', 'Japanese Restaurant']
----------------------------------------------------------------------------------
Label 1 : blue
Top-5 Categories:
 ['Park', 'Trail', 'Tennis Court', 'Gym', 'Jewelry Store']
----------------------------------------------------------------------------------
Label 2 : yellow
Top-5 Categories:
 ['Park', 'Playground', 'Trail']
----------------------------------------------------------------------------------
Label 3 : brown
Top-5 Categories:
 ['Home Service', 'Garden']
----------------------------------------------------------------------------------
Label 4 : red
Top-5 Categories:
 ['Bus Line', 'Park', 'Swim School']
----------------------------------------------------------------------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
